# Set environment settings :: Library dependencies

In [41]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly as py
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px

import pandas as pd
from pandas_datareader import data as web
import numpy as np
from datetime import datetime
import datetime

# To grab stock data
import yfinance as fyf
from pandas_datareader import data as pdr
fyf.pdr_override() # <-- Here is the fix

# Load in datasets into Data Frame

In [42]:
unit = pd.read_csv('dummy_cost_unit.csv')
unit.set_index('date', inplace=True)

div_table = unit[unit.type.str.contains('Dividend')] ## Dividend records only

Let's create separate tables for each individual fund

In [43]:
puritan = unit[unit.symbol.str.contains('FPURX')] ## Each symbol, both purchases and dividends
well = unit[unit.symbol.str.contains('VWELX')]
spy = unit[unit.symbol.str.contains('SPY')]
agg = unit[unit.symbol.str.contains('AGG')]

Let's create separate tables for each individual fund, dividends only.

In [44]:
puritan_d = puritan[puritan.type.str.contains('Dividend')]  ## Each symbol, dividends only
spy_d = spy[spy.type.str.contains('Dividend')]
well_d = well[well.type.str.contains('Dividend')]
agg_d = agg[agg.type.str.contains('Dividend')]

The following code pulls the newest data from the Yahoo Finance API

In [45]:
sample = ["AGG","VWELX","FPURX","ONEQ","SPY"]

# Set start and end dates
start = datetime.datetime(2015, 1, 1)
end   = datetime.datetime(2020, 4, 21)

df1 = pdr.get_data_yahoo(sample, start = start, end = end)

[*********************100%***********************]  5 of 5 completed


We'll narrow down just the daily Close of each fund

In [46]:
df2 = df1['Close']

In [47]:
df2

,AGG,FPURX,ONEQ,SPY,VWELX
Date,,,,,
2015-01-02,110.430000,21.490000,184.899994,205.429993,39.160000
2015-01-05,110.669998,21.230000,183.130005,201.720001,38.730000
2015-01-06,110.949997,21.080000,179.899994,199.820007,38.599998
2015-01-07,110.930000,21.280001,182.160004,202.309998,38.840000
2015-01-08,110.760002,21.530001,185.880005,205.899994,39.259998
...,...,...,...,...,...
2020-04-14,116.760002,21.490000,330.679993,283.790009,39.490002
2020-04-15,117.239998,21.290001,326.890015,277.760010,38.930000
2020-04-16,117.389999,21.490000,332.970001,279.100006,39.099998


# Prepare queries within Pandas

In [48]:
unit

,quarter,account,symbol,type,num_shares,share_price,principal,sym_total,run_share2,dividend
date,,,,,,,,,,
2015-01-28,2015 Q1,Taxable,AGG,Stock Purchase,0.24,112.07,26.70,26.70,0.24,NaN
2015-02-02,2015 Q1,Taxable,AGG,Dividend,0.00,112.20,0.04,26.74,0.24,0.18
2015-02-11,2015 Q1,Taxable,AGG,Stock Purchase,0.60,110.87,66.75,93.49,0.84,NaN
2015-02-25,2015 Q1,Taxable,AGG,Stock Purchase,0.60,111.32,66.75,160.24,1.44,NaN
2015-03-02,2015 Q1,Taxable,AGG,Dividend,0.00,110.45,0.27,160.51,1.44,0.19
...,...,...,...,...,...,...,...,...,...,...
2020-03-10,2020 Q1,Taxable,VWELX,Stock Purchase,5.28,40.49,213.60,11848.70,290.08,NaN
2020-03-11,2020 Q1,Taxable,VWELX,Stock Purchase,3.84,39.10,150.00,11998.70,293.91,NaN
2020-03-20,2020 Q1,Taxable,VWELX,Dividend,2.37,33.62,79.60,12078.30,295.90,0.27


Let's summarize each symbol's cost basis

In [49]:
df4 = unit.groupby(['symbol'], as_index=False)['share_price'].mean()

In [50]:
df4

,symbol,share_price
0,AGG,109.590885
1,FPURX,21.689735
2,SPY,245.921400
3,VWELX,40.043733


Create a simple Plotly graph to visualize the cost basis

In [51]:
fig5 = go.Figure(go.Bar(
    x=df4['symbol'],  # NOC stands for National Olympic Committee
    y=df4['share_price']))

In [52]:
fig5

Summarize each symbol by quarterly performance

In [53]:
df3 = unit.groupby(['symbol','quarter'], as_index=False)['principal'].sum()
print (df3)

agg_q = df3[df3.symbol.str.contains('AGG')]
spy_q = df3[df3.symbol.str.contains('SPY')]
well_q = df3[df3.symbol.str.contains('VWELX')]
pur_q = df3[df3.symbol.str.contains('FPURX')]

   symbol  quarter  principal
0     AGG  2015 Q1     294.01
1     AGG  2015 Q2     438.53
2     AGG  2015 Q3     548.48
3     AGG  2015 Q4     588.32
4     AGG  2016 Q1     569.54
..    ...      ...        ...
83  VWELX  2019 Q2     349.07
84  VWELX  2019 Q3     303.20
85  VWELX  2019 Q4     655.40
86  VWELX  2020 Q1     586.00
87  VWELX  2020 Q2      50.00

[88 rows x 3 columns]


Visualize each fund by the amount of dividends returned in the portfolio

In [72]:
import plotly.graph_objects as go

fig1 = go.Figure(go.Bar(x=agg_d.quarter, y=agg_d.principal, name='AGG'))
fig1.add_trace(go.Bar(x=spy_d.quarter, y=spy_d.principal, name='SPY'))
fig1.add_trace(go.Bar(x=well_d.quarter, y=well_d.principal, name='Wellington'))
fig1.add_trace(go.Bar(x=puritan_d.quarter, y=puritan_d.principal, name='Puritan'))

fig1.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig1.show()

In [73]:
import plotly.graph_objects as go

fig2 = go.Figure(go.Bar(x=agg_q.quarter, y=agg_q.principal, name='AGG'))
fig2.add_trace(go.Bar(x=spy_q.quarter, y=spy_q.principal, name='SPY'))
fig2.add_trace(go.Bar(x=well_q.quarter, y=well_q.principal, name='Wellington'))
fig2.add_trace(go.Bar(x=pur_q.quarter, y=pur_q.principal, name='Puritan'))

fig2.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig2.show()

Let's start a Waterfall graph that we can later fill in with monthly or quarterly profit and loss summaries

In [75]:
import plotly.graph_objects as go

fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = ["relative", "relative", "total", "relative", "relative", "total"],
    x = ["Sales", "Consulting", "Net revenue", "Purchases", "Other expenses", "Profit before tax"],
    textposition = "outside",
    text = ["+60", "+80", "", "-40", "-20", "Total"],
    y = [60, 80, 0, -40, -20, 0],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Profit and loss statement 2018",
        showlegend = True
)

fig.show()

# Account principal over time

In [76]:
div_qtr = div_table.groupby('quarter')['principal'].sum()

# Average share price for each mutual fund / ETF / stock :: Cost Basis

In [77]:
cost_basis = unit.groupby('symbol')['share_price'].mean()

In [78]:
cost_basis

symbol
AGG      109.590885
FPURX     21.689735
SPY      245.921400
VWELX     40.043733
Name: share_price, dtype: float64

In [79]:
spy['sym_total'].head(80)

date
2015-01-28      26.70
2015-02-11      93.45
2015-02-25     160.20
2015-03-11     226.95
2015-03-20     227.98
               ...   
2017-07-26    5186.23
2017-08-09    5227.33
2017-08-23    5268.42
2017-09-06    5309.52
2017-09-15    5340.53
Name: sym_total, Length: 80, dtype: float64

# Create list of charts for inputs in the App

Create the stacked area chart to visualize fund performance over time.

In [80]:
fig = go.Figure()

In [81]:
fig.add_trace(go.Scatter(
    x=agg['quarter'], y=agg['sym_total'],
    name="Taxable: AGG",
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(0, 51, 153)'),
    stackgroup='one' # define stack group
))
fig.add_trace(go.Scatter(
    x=puritan['quarter'], y=puritan['sym_total'],
    name="Taxable: Puritan Fund",
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(131, 90, 241)'),
    stackgroup='one' # define stack group
))
fig.add_trace(go.Scatter(
    x=well['quarter'], y=well['sym_total'],
    name="Taxable: Wellsley",
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(102, 0, 204)'),
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    x=spy['quarter'], y=spy['sym_total'],
    name="Taxable: SPY",
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(50, 0, 204)'),
    stackgroup='one'
))

fig.update_layout(yaxis_range=(0, 55000))

Create a treemap to show symbol summary

In [84]:
fig4 = px.treemap(unit, path=['symbol'], values='principal')

In [85]:
fig4

In [66]:
puritan_d.index

Index(['2015-04-01', '2015-07-01', '2015-10-09', '2015-12-11', '2016-04-08',
       '2016-07-08', '2016-10-14', '2016-12-09', '2016-12-27', '2017-04-07',
       '2017-07-07', '2017-10-13', '2017-12-08', '2018-04-06', '2018-07-06',
       '2018-10-12', '2018-12-07', '2019-04-05', '2019-07-05', '2019-10-11',
       '2019-12-13', '2020-04-03'],
      dtype='object', name='date')

# Prepare series of each symbol

In [86]:
agg_sp = agg['share_price']
spy_sp = spy['share_price']
pure_sp = puritan['share_price']
well_sp = well['share_price']

In [87]:
agg_sp

date
2015-01-28    112.07
2015-02-02    112.20
2015-02-11    110.87
2015-02-25    111.32
2015-03-02    110.45
               ...  
2020-03-10    116.38
2020-03-11    114.72
2020-03-25    114.73
2020-04-01    114.73
2020-04-08    115.81
Name: share_price, Length: 192, dtype: float64

# Layout and Data :: Plotly and Dash Application

In [90]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Cost Basis: Taxable', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(
                            x = df2.index,
                            y = df2['SPY'],
                            mode = 'lines',
                            name='SPY Daily Close',
                            marker = dict(color='#D7DF01')
                        ),
                        go.Scatter(
                            x = spy_sp.index,
                            y = spy_sp,
                            mode = 'markers',
                            name='Cost Basis',
                            marker = dict(color='black',size=3)
                        ),
                        go.Scatter(
                            x = df2.index,
                            y = df2['SPY'].rolling(50).mean(),
                            mode = 'lines',
                            name='MA 50',
                            marker=dict(
                                color='gray',
                                line = dict(width=6)
                            )
                        )
                    ],
                'layout': go.Layout(
                    title = 'SPDR S&P 500 ETF Trust Daily Close 2015 - present',
                    xaxis = {'title': 'Time Interval (daily)'},
                    yaxis = {'title': 'Price per Share ($/share)'},
                    hovermode='closest'
                )
            }
        ),
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(
                            x = df2.index,
                            y = df2['FPURX'],
                            mode = 'lines',
                            name='FPURX Daily Close',
                            marker = dict(color='#D7DF01')
                        ),
                        go.Scatter(
                            x = pure_sp.index,
                            y = pure_sp,
                            mode = 'markers',
                            name='Cost Basis',
                            marker = dict(color='black',size=3)
                        ),
                        go.Scatter(
                            x = df2.index,
                            y = df2['FPURX'].rolling(50).mean(),
                            mode = 'lines',
                            name='MA 50',
                            marker=dict(
                                color='gray',
                                line = dict(width=6)
                            )
                        )
                    ],
                'layout': go.Layout(
                    title = 'Fidelity Puritan Mutual Fund Daily Close 2015 - present',
                    xaxis = {'title': 'Time Interval (daily)'},
                    yaxis = {'title': 'Price per Share ($/share)'},
                    hovermode='closest'
                )
            }
        ),
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(
                            x = df2.index,
                            y = df2['VWELX'],
                            mode = 'lines',
                            name='FSGGX Daily Close',
                            marker = dict(color='#D7DF01')
                        ),
                        go.Scatter(
                            x = well_sp.index,
                            y = well_sp,
                            mode = 'markers',
                            name='Cost Basis',
                            marker = dict(color='black',size=3)
                        ),
                        go.Scatter(
                            x = df2.index,
                            y = df2['VWELX'].rolling(50).mean(),
                            mode = 'lines',
                            name='MA 50',
                            marker=dict(
                                color='gray',
                                line = dict(width=6)
                            )
                        )
                    ],
                'layout': go.Layout(
                    title = 'Vanguard Wellington Mutual Fund Daily Close 2015 - present',
                    xaxis = {'title': 'Time Interval (daily)'},
                    yaxis = {'title': 'Price per Share ($/share)'},
                    hovermode='closest'
                )
            }
        ),
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(
                            x = df2.index,
                            y = df2['AGG'],
                            mode = 'lines',
                            name='AGG Daily Close',
                            marker = dict(color='#D7DF01')
                        ),
                        go.Scatter(
                            x = agg_sp.index,
                            y = agg_sp,
                            mode = 'markers',
                            name='Cost Basis',
                            marker = dict(color='black',size=3)
                        ),
                        go.Scatter(
                            x = df2.index,
                            y = df2['AGG'].rolling(50).mean(),
                            mode = 'lines',
                            name='MA 50',
                            marker=dict(
                                color='gray',
                                line = dict(width=6)
                            )
                        )
                    ],
                'layout': go.Layout(
                    title = 'iShares Barclays Aggregate Bond Fund Daily Close 2015 - present',
                    xaxis = {'title': 'Time Interval (daily)'},
                    yaxis = {'title': 'Price per Share ($/share))'},
                    hovermode='closest'
                )
            }
        ),
            dcc.Graph(
                figure={
                    'data': [
                        go.Bar(
                        x=df4['symbol'],
                        y=df4['share_price']
                        )
                    ],                
                    'layout': go.Layout(
                        title = 'Average Cost Basis',
                        xaxis = {'title': 'Fund or Symbol'},
                        yaxis = {'title': 'Price per Share'},
                        hovermode='closest'
                    )  
                }
            )
    ]),
        dcc.Tab(label='Performance', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(
                            x = agg['quarter'],
                            y = agg['sym_total'],
                            name="Taxable: AGG",
                            hoverinfo='x+y',
                            mode='lines',
                            line=dict(width=0.5, color='rgb(0, 51, 153)'),
                            stackgroup='one' # define stack group
                        ),
                        go.Scatter(
                            x = puritan['quarter'],
                            y = puritan['sym_total'],
                            name="Taxable: Puritan Fund",
                            hoverinfo='x+y',
                            mode='lines',
                            line=dict(width=0.5, color='rgb(131, 90, 241)'),
                            stackgroup='one' # define stack group
                        ),
                        go.Scatter(
                            x = well['quarter'],
                            y = well['sym_total'],
                            name="Taxable: Wellsley",
                            hoverinfo='x+y',
                            mode='lines',
                            line=dict(width=0.5, color='rgb(102, 0, 204)'),
                            stackgroup='one'
                        ),
                        go.Scatter(
                            x = spy['quarter'],
                            y = spy['sym_total'],
                            name="Taxable: SPY",
                            hoverinfo='x+y',
                            mode='lines',
                            line=dict(width=0.5, color='rgb(50, 0, 204)'),
                            stackgroup='one'
                            )
                    ],
                'layout': go.Layout(
                    title = 'Portfolio Performance over Time',
                    xaxis = {'title': 'Time Interval (daily)'},
                    yaxis = {'title': 'Price per Share ($/share)'},
                    hovermode='closest',
                )
            }
        ),
            dcc.Graph(
                figure={
                    'data': [
                        go.Bar(
                            x = well_d.quarter,
                            y = well_d.principal,
                            name='Wellseley'
                        ),
                        go.Bar(
                            x = puritan_d.quarter,
                            y = puritan_d.principal,
                            name='Puritan'
                        ),
                        go.Bar(
                            x = agg_d.quarter,
                            y = agg_d.principal,
                            name='AGG'
                        ),
                        go.Bar(
                            x = spy_d.quarter,
                            y = spy_d.principal,
                            name='SPY'
                        ),
                    ],
                'layout': go.Layout(
                    title = 'Quarterly Dividends & Capital Gains Reinvested back into the Portfolio by Fund',
                    xaxis = {'title': 'Time Interval (quarterly)'},
                    yaxis = {'title': 'Principal Amount'},
                    hovermode='closest',
                    barmode='stack'
                )
            }
        ),
            dcc.Graph(
                figure={
                    'data': [
                        go.Bar(
                            x = pur_q.quarter,
                            y = pur_q.principal,
                            name='Puritan'
                        ),
                        go.Bar(
                            x = well_q.quarter,
                            y = well_q.principal,
                            name='Wellseley'
                        ),
                        go.Bar(
                            x = spy_q.quarter,
                            y = spy_q.principal,
                            name='SPY'          
                        ),
                        go.Bar(
                            x = agg_q.quarter,
                            y = agg_q.principal,
                            name='AGG',
                        ),
                    ],
                'layout': go.Layout(
                    title = 'Quarterly Contributions by Fund',
                    xaxis = {'title': 'Time Interval (quarterly)'},
                    yaxis = {'title': 'Principal Amount'},
                    hovermode='closest',
                    barmode='stack'
                )
            }
        ),
        ]),
        dcc.Tab(label='Free Cash Flow', children=[
            dcc.Graph(
                figure={
                    'data': [
                        go.Waterfall(
                            name = "Change over Time",
                            orientation = "v",
                            measure = ["total", "relative", "relative", "relative", "relative", "total"],
                            x = ["Oct 2019", "Nov 2019", "Dec 2019", "Jan 2020", "Feb 2020","Mar 2020","Apr 2020"],
                            textposition = "outside",
                            text = ["+60", "+80", "", "-40", "-20", "Total"],
                            y = [50754, -542.08, -9193.78, 1043.48, 64.54, -798.42, 43340.99],
                            connector = {"line":{"color":"rgb(63, 63, 63)"}},
                        )
                    ],
                'layout': go.Layout(
                    title = 'Profit and Loss Statement in 2020',
                    showlegend = True
                    )
                }
            ),
        ]),
    ])
])

# Call Application

In [ ]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2020 13:51:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2020 13:51:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2020 13:51:42] "GET /_favicon.ico?v=1.11.0 HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2020 13:51:42] "GET /_dash-layout HTTP/1.1" 200 -
